# Figures present in the paper:

   - 1- figure of the proof of bounding truncated force from $A^{(0,1)}$ 
   - 2- Simulations figures homogeneous case 
   - 3- Variance reduction with respect to epsilon

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=8)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from structure_factor.spatial_windows import BallWindow, BoxWindow
from structure_factor.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravitational_force import force_k
from GPPY.gravity_point_process import GravityPointProcess
import copy
import numpy as np

# 1- Figure of Section 3: 

# Simulation of push

In [ ]:
d = 2
r=1
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 2000
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
gppy = GravityPointProcess(poisson_pp)
push_pp = gppy.pushed_point_pattern()


In [ ]:
#finding the force on each point
points = poisson_pp.points
nb_points = points.shape[0]
force_x, force_y=[],[]
epsilon = gppy.epsilon_critical
for k in range(0,nb_points):
    #force_ = epsilon*force_k(k=k, x=points[k], point_pattern=copy.deepcopy(poisson_pp))
    force = force_k(k=k, x=points[k], point_pattern=copy.deepcopy(poisson_pp))[0].tolist()
    force_x.append(force[0])
    force_y.append(force[1])


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
simu_window.plot(axis=ax[0], color="grey")
poisson_pp.plot(axis=ax[0], s=0.1)
poisson_pp.plot(axis=ax[1], c="g", s=0.1, label="Poisson")
ax[1].quiver(points[:,0], points[:,1], force_x, force_y, units='xy')
push_pp.plot(axis=ax[2], s=0.1)
simu_window.plot(axis=ax[2], color="grey")
ax[1].legend()
plt.savefig("poisson_and_push_and_force.pdf")
plt.show()

## 2 -  Figure of variance redutcion 

In [ ]:
import numpy.random as npr
# Setting a seed for reproducibility
npr.seed(3)

def norm_x(x) :
    return np.linalg.norm(x, axis=1)**2

def f_1(x) :
    return 1- np.linalg.norm(x, axis=1)**2

def f_2(x):
    return np.sum(x, axis=1)/(norm_x(x) +1)

def f_3(x):
    return -3*np.cos(norm_x(x)) + 4*np.sin(f_2(x))

def bump_indicator(points, n=10, R=4):
    norm_points = np.linalg.norm(points, axis=1)
    f = np.zeros((points.shape[0]))
    f[norm_points<R]= np.exp(-1/(n*(1- (norm_points[norm_points<R]/R)**2)))
    return f


In [ ]:
r=4 #radius observation window
x = np.linspace(-r,r, 40)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
z_f_4 = bump_indicator(points)
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1)
ax.set_title("f_1")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2)
ax.set_title("f_2")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_2)
ax.set_title("f_3")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4)
ax.set_title("Bump indicator")
plt.show()

In [ ]:
from GPPY.numerical_integration import monte_carlo_integration
import statistics as stat

def var_lin_stat(f, points_list):
    mc_values = [monte_carlo_integration(points=p, f=f) for p in points_list]
    mean_mc = stat.mean(mc_values)
    std_mc = stat.stdev(mc_values)
    return mean_mc, std_mc
    

d=3

In [ ]:
intensity

In [ ]:
# F truncated is denoted F tilde, error notation
from structure_factor.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravity_point_process import GravityPointProcess
from structure_factor.spatial_windows import BallWindow

# Creat a poisson point process
d = 3
samples_nb = 40
nb_points =2000
R=8 #simulation radius
r= 4 #radius observation window
means_f1_push, stds_f1_push, means_f2_push, stds_f2_push= [], [], [], []
means_f3_push, stds_f3_push, means_f4_push, stds_f4_push = [], [], [], [] 

#poisson pp
intensity = nb_points/window.volume
simu_window = BallWindow(center=[0]*d, radius=R)
window = BallWindow(center=[0]*d, radius=r)
print("expected points_nb =", nb_points, 
      "mean points_nb_simu=", intensity* simu_window.volume)
## list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp_big = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]

#Variance for poisson
poisson_pp = [p.restrict_to_window(window) for p in poisson_pp_big]
poisson_pts_list = [p.points for p in poisson_pp]
poisson_mean_nb_pt = stat.mean(p.shape[0] for p in poisson_pts_list)
## for f_1, f_2, f_3, f_4
mean_f1_poisson, std_f1_poisson = var_lin_stat(f_1, poisson_pts_list)
mean_f2_poisson, std_f2_poisson = var_lin_stat(f_2, poisson_pts_list)
mean_f3_poisson, std_f3_poisson = var_lin_stat(f_3, poisson_pts_list)
mean_f4_poisson, std_f4_poisson = var_lin_stat(bump_indicator, poisson_pts_list)

#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
epsilon_0 = gpp_pp[0].epsilon_critical
print("d=", d, ",Nb samples=", samples_nb, ",Epsilon critical=", epsilon_0)
epsilons = [-0.05, -0.03, -0.01, -epsilon_0, -0.004, -0.001, -0.0005,
            0.001, 0.004, 0.005, epsilon_0, 0.006, 0.007, 0.008, 
            0.009, 0.01, 0.04, 0.07]
t=0
print("----------------------------------------------")
for epsilon in epsilons:
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
        
        #using F
        push_pp_big = [g.pushed_point_pattern(epsilon=epsilon) for g in gpp_pp]
        push_pp = [g.restrict_to_window(window) for g in push_pp_big]
        push_pts_list = [g.points for g in push_pp]
        push_mean_nb_pt = stat.mean(g.shape[0] for g in push_pts_list)
        
        if t==0:
            time_end = time.time() - time_start
            print("start_time", int(time_end/60), "min", time_end%60, "s")
            print("----------------------------------------------")
            
        #plot 
        fig, axis = plt.subplots(1, 2, figsize=(15, 5))
        push_pp_big[0].plot(window=simu_window, axis= axis[0], c="r", s=0.2)
        push_pp[0].plot(window=simu_window, axis= axis[0], c="b", s=0.2)
        poisson_pp_big[0].plot(window=simu_window, axis= axis[1], c="r", s=0.2)
        poisson_pp[0].plot(window=simu_window, axis= axis[1], c="b", s=0.2)
        plt.show()
        
        # mean number of points in the restricted window
        print("mean nb_pts poisson", poisson_mean_nb_pt, 
              "mean nb_pts push", push_mean_nb_pt,
             )
        # variance for push
        ## f_1
        mean_f, std_f = var_lin_stat(f_1, push_pts_list)
        means_f1_push.append(mean_f)
        stds_f1_push.append(std_f)
        ## f_2
        mean_f, std_f = var_lin_stat(f_2, push_pts_list)
        means_f2_push.append(mean_f)
        stds_f2_push.append(std_f)
        ## f_3
        mean_f, std_f = var_lin_stat(f_3, push_pts_list)
        means_f3_push.append(mean_f)
        stds_f3_push.append(std_f)
        ## f_4
        mean_f, std_f = var_lin_stat(bump_indicator, push_pts_list)
        means_f4_push.append(mean_f)
        stds_f4_push.append(std_f)
        
        
        
        print("FOR f_1:")
        print("std poisson", std_f1_poisson, ",std push", stds_f1_push)
        print("----------------------------------------------")
        
        print("FOR f_2:")
        print("std poisson", std_f2_poisson, ",std push", stds_f2_push)
        print("----------------------------------------------")
        
        print("FOR f_3:")
        print("std poisson f_3", std_f3_poisson, ",std push", stds_f3_push)
        print("----------------------------------------------")
        
        print("FOR f_4:")
        print("std poisson f_4", std_f4_poisson, ",std push", stds_f4_push)
        print("----------------------------------------------")
        
        t=1
time_final = time.time() - time_start
print("start_time", int(time_final/60), "min", time_final%60, "s")


In [ ]:
fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_2)
ax.set_title(r"$f_3$")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4)
ax.set_title(r"$f_4$")
plt.savefig("linear_stat_used_for_var_comp.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 4))
#f_1
ax[0].plot(epsilons, stds_f1_push, "k.", label=r"Push" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].plot(0, std_f1_poisson, "r*", label="Poisson")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push), ymax=max(stds_f1_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
#ax[0].set_title(r"$f_1$")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma$")
#f_2
ax[1].plot(epsilons, stds_f2_push,  "k.", label=r"Push")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(0, std_f2_poisson, "r*", label="Poisson")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push), ymax=max(stds_f2_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push,  "k.", label=r"Push")
ax[2].plot(epsilons, stds_f3_push,  "k")
ax[2].plot(0, std_f3_poisson, "r*", label="Poisson")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push), ymax=max(stds_f3_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")

ax[2].legend()
#ax[2].set_title(r"$f_3$")
ax[2].set_xlabel(r"$\epsilon$")
#ax[2].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[3].plot(epsilons, stds_f4_push,  "k.", label=r"Push")
ax[3].plot(epsilons, stds_f4_push,  "k")
ax[3].plot(0, std_f4_poisson, "r*", label="Poisson")
ax[3].vlines(epsilon_0, ymin=min(stds_f4_push), ymax=max(stds_f4_push), colors="grey", 
             linestyles='dashed', label=r"$\epsilon_0$")

ax[3].legend()
#ax[3].set_title(r"$f_3$")
ax[3].set_xlabel(r"$\epsilon$")
#ax[3].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
plt.savefig("variance_linear_stat_3d.pdf")
plt.show()

# 3-figure of the proof of bounding truncated force from $A^{(0,1)}$

In [ ]:
from structure_factor.spatial_windows import BallWindow
def sphere_number(p,q, d):
    r = (p-q)/2
    window = BallWindow(center= [0]*d, radius=q + r)
    return window.surface/(2*r**(d-1))
def next_points(a_0, p, q):
    r = (p-q)/2
    x_1 = (q**2-r**2 + 2*q*r)/(q+r)
    return np.array([[x_1, np.sqrt((q+r)**2 - x_1**2)], [x_1, - np.sqrt((q+r)**2 - x_1**2)]])
def next_points_2(a, p, q):
    r = (p-q)/2
    b_ = (4*r**2*a[1]**2 - 2*a[1]*(q+r)**2)
    a_ = a[1]**2 + a[0]**2 
    c_ = (q + r)**4 + 4*r**4 - 4*r**2*(q + r)**2 - a[0]**2*(q+r)**2
    delta = b_**2 - 4*a_*c_
    print(delta)
    x_2 = [(-b_-np.sqrt(delta))/(2*a_), (-b_+np.sqrt(delta))/(2*a_)]
    print(x_2)
    x_1 = [((q+r)**2 - 2*r**2 - x_2[0]*a[1])/a[1], ((q+r)**2 - 2*r**2 - x_2[1]*a[1])/a[1]]
    return np.array([[x_1[0], x_2[0]], [x_1[1], x_2[1]]]) 

In [ ]:
import numpy as np

epsilon = 0.5
x = np.array([3,76])
a_0 = [76, 0]
d=2
origin = [0]*d
norm_x = np.linalg.norm(x)
R=8
r=R/epsilon
p, q = norm_x + r, norm_x - r
sphere_nb = sphere_number(p=p, q=q, d=d)
points_next = next_points(a_0, p, q)
print(np.linalg.norm(points_next, axis=1))


In [ ]:

print("sphere number=", sphere_nb)
window0 = BallWindow(center=origin, radius=norm_x)
window1 = BallWindow(center=origin, radius=R)
window2 = BallWindow(center=origin, radius=norm_x-R/epsilon)
window3 = BallWindow(center=origin, radius=norm_x + R/epsilon)
window4 = BallWindow(center=x, radius=R/epsilon)
window5 = BallWindow(center=[np.sqrt((q+r)**2 - (q/(q+r))**2), q/(q+r)], radius=r)
window6 = BallWindow(center=points_next[0,:], radius=r)
window7 = BallWindow(center=points_next[1,:], radius=r)

In [ ]:
import numpy as np
from structure_factor.spatial_windows import BallWindow

theta = np.arcsin(r/norm_x)
alpha = np.linspace(-theta, theta, 100)
u, v = (R+5)*np.cos(alpha), (R+5)*np.sin(alpha)
nb_theta = int(180/(theta*57.2958)/2)
print(nb_theta)

fig, ax = plt.subplots(figsize=(4,4))
window3.plot(axis=ax, color="b", linestyle="--")
window2.plot(axis=ax, color="b", linestyle="--")
ax.plot(0, 0, "k.")
plt.plot(a_0[0], a_0[1], 'r.')
plt.fill_between([norm_x-r,np.cos(theta)*(norm_x+r)], 
                 [np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x+r)], 
                 [-np.sin(theta)*(norm_x-r), -np.sin(theta)*(norm_x+r)], facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x-r))
plt.fill_betweenx(y_, np.sqrt((norm_x -r)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x), np.sin(theta)*(norm_x))
plt.fill_betweenx(y_, np.sqrt((norm_x)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
for i in range(0, nb_theta ):
    a,b= np.cos(2*i*theta)*norm_x, np.sin(2*i*theta)*norm_x
    window = BallWindow(center=[a,b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, b, 'r.')
    window = BallWindow(center=[a,-b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, -b, 'r.')
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [np.sin((2*i+1)*theta)*(norm_x-r), np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [-np.sin((2*i+1)*theta)*(norm_x-r), -np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
c, d = np.cos(2*7*theta)*norm_x, np.sin(2*7*theta)*norm_x
window = BallWindow(center=[c,d ], radius=r)
window.plot(axis=ax, color="k")
plt.plot(c, d, 'r.')
plt.plot([0, c], [0, d], 'grey', linestyle="--")
plt.gca().annotate(r'$\|\mathbf{x}\|_2$', xy=(c/2, d -5), xycoords='data', fontsize=8, rotation=-10)
plt.plot([0,  np.cos(theta)*(norm_x+r)], 
        [0, np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([0, np.cos(theta)*(norm_x+r)], 
        [0, -np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([np.cos(15*theta)*(norm_x-r), np.cos(15*theta)*(norm_x+r)], 
             [np.sin((15)*theta)*(norm_x-r), np.sin((15)*theta)*(norm_x+r)], 'b' )
plt.plot([norm_x-r,norm_x],[0, 0], 'k' )
plt.gca().annotate(r'$f(\mathbf{x})$', xy=(norm_x -15, 3), xycoords='data', fontsize=8)
plt.plot([0,-8],[0, 0], 'k' )
plt.gca().annotate(r'$R$', xy=(-6, -6), xycoords='data', fontsize=7)
window1.plot(axis=ax, color="k")
window0.plot(axis=ax, color="grey", linestyle="-")
plt.plot(u,v, "grey")
plt.gca().annotate(r'$\theta$', xy=(R+7, -2), xycoords='data', fontsize=8)
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.savefig("proof_existance_pic.pdf")
plt.show()


# Section 3

## 4-  Sampling figures

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import timeit
import math

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

## d=2

In [ ]:
d = 2
l=1
obs_window = BoxWindow([[-l/2, l/2]]*d) # observation window
diam_obs_window = l*math.sqrt(d)
#simulation window push with F
simu_window_push1 = BallWindow(center=[0]*d, radius=diam_obs_window)
#simulation window push with F truncated
simu_window_push2 = BallWindow(center=[0]*d, radius=3*diam_obs_window/2)
window_pushed_points = BallWindow(center=[0]*d, radius=diam_obs_window/2)

intensity = 1000
points_nb = intensity*simu_window_push1.volume 
print("points_nb=", points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window_push2, seed=0) 
points=poisson_pp.points
poisson_pp.plot()

In [ ]:
gpp1 = GravityPointProcess(poisson_pp.restrict_to_window(simu_window_push1))
epsilon_critical = gpp1.epsilon_critical
print("critical epsilon=", epsilon_critical)
start = timeit.default_timer()
if __name__=='__main__':
    push1_pp = gpp1.pushed_point_pattern(epsilon=epsilon_critical)
time_end = timeit.default_timer() - start
print("time=", int(time_end/60), "min", time_end%60, "s")


In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
push1_pp.plot(axis=ax)
simu_window_push1.plot(axis=ax,color="r")
window_pushed_points.plot(axis=ax, color="b")
plt.show()

In [ ]:
# push with truncated force
gpp2 = GravityPointProcess(poisson_pp)
epsilon_critical = gpp2.epsilon_critical
start = timeit.default_timer()
if __name__=='__main__':
    push2_pp = gpp2.pushed_point_pattern(epsilon=epsilon_critical, 
                                         p=diam_obs_window, correction=False
                                        ) 
time_end = timeit.default_timer() - start
print("time=", int(time_end/60), "min", time_end%60, "s")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
push2_pp.plot(axis=ax)
simu_window_push1.plot(axis=ax,color="r")
window_pushed_points.plot(axis=ax, color="b")
plt.show()

In [ ]:

fig, axis = plt.subplots(1, 4, figsize=(10, 5))
poisson_pp.plot(window=obs_window, axis=axis[0], c="b", s=0.2)
axis[0].set_title("Poisson sample")
push1_pp.plot(window=obs_window, axis=axis[1], c="k", s=0.2)
axis[1].set_title(r"Pushed sample using $(F_2)$")
push2_pp.plot(window=obs_window, axis=axis[2], c="k", s=0.2)
axis[2].set_title(r"Pushed sample using $F^{(0, 2)}$")
push1_pp.plot(window=obs_window, axis=axis[3], c="k", s=0.2, label=r"$(F_2)$")
push2_pp.plot(window=obs_window, axis=axis[3], c="g", s=0.2, label=r"$F^{(0, 2)}$")
axis[3].set_title(r"Pushed samples")
axis[3].legend()
plt.tight_layout()
plt.savefig("poisson_and_push_2D.pdf", bbox_inches='tight')
plt.show()

# d=3

In [ ]:
d = 3
l=1
obs_window = BoxWindow([[-l/2, l/2]]*d) # observation window
diam_obs_window = l*math.sqrt(d)
#simulation window push with F
simu_window_push1 = BallWindow(center=[0]*d, radius=diam_obs_window)
#simulation window push with F truncated
simu_window_push2 = BallWindow(center=[0]*d, radius=3*diam_obs_window/2)
window_pushed_points = BallWindow(center=[0]*d, radius=diam_obs_window/2)
 

intensity = 1000
points_nb = intensity*simu_window_push1.volume 
print("points_nb=", points_nb, "nb_simu_points=", intensity*simu_window_push2.volume)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window_push2, seed=0) 
points=poisson_pp.points
#poisson_pp.plot()

In [ ]:
gpp1 = GravityPointProcess(poisson_pp.restrict_to_window(simu_window_push1))
epsilon_critical = gpp1.epsilon_critical
print("critical epsilon=", epsilon_critical)
start = timeit.default_timer()
if __name__=='__main__':
    push1_pp = gpp1.pushed_point_pattern(epsilon=epsilon_critical)
time_end = timeit.default_timer() - start
print("time=", int(time_end/60), "min", time_end%60, "s")
#push1_pp.plot()


In [ ]:
# push with truncated force
gpp2 = GravityPointProcess(poisson_pp)
epsilon_critical = gpp2.epsilon_critical
start = timeit.default_timer()
if __name__=='__main__':
    push2_pp = gpp2.pushed_point_pattern(epsilon=epsilon_critical, 
                                         p=diam_obs_window, correction=False
                                        ) 
time_end = timeit.default_timer() - start
print("time=", int(time_end/60), "min", time_end%60, "s")
#push2_pp.plot()

In [ ]:
push1_points = push1_pp.restrict_to_window(obs_window).points
push2_points = push2_pp.restrict_to_window(obs_window).points
points = poisson_pp.restrict_to_window(obs_window).points

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], s=1, cmap='Blues')
ax.view_init(40, 60)
#ax.set_title(r"Poisson sample")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(push1_points[:, 0], push1_points[:, 1], push1_points[:, 2], 
             c=push1_points[:, 2], s=1, cmap='Greys')
ax.view_init(40, 60)
#ax.set_title(r"Pushed sample using $(F_2)$")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(push2_points[:, 0], push2_points[:, 1], push2_points[:, 2], 
             c=push2_points[:, 2], s=1, cmap='Greys')
ax.view_init(40, 60)
#ax.set_title(r"Push sample using $F^{(0, 2)}$")
#ax.set_title(r"Pushed sample using $(F_2)$")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(push1_points[:, 0], push1_points[:, 1], push1_points[:, 2], 
             c=push1_points[:, 2], s=1, cmap='Greys', label=r"$(F_2)$")
ax.scatter3D(push2_points[:, 0], push2_points[:, 1], push2_points[:, 2], 
             c=push2_points[:, 2], s=1, cmap='Greens', label=r" $F^{(0, 2)}$")
ax.legend()
ax.view_init(40, 60)
plt.tight_layout()
plt.savefig("poisson_and_push_3D.pdf", bbox_inches='tight')
plt.show()

# 3- Variance reduction 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy.random as npr
# Setting a seed for reproducibility
npr.seed(3)

In [ ]:
import numpy.random as npr
# Setting a seed for reproducibility
npr.seed(3)

def norm_x(x) :
    return np.linalg.norm(x, axis=1)**2

def f_1(x) :
    return 1- np.linalg.norm(x, axis=1)**2

def f_2(x):
    return np.sum(x, axis=1)/(norm_x(x) +1)

def f_3(x):
    return -3*np.cos(norm_x(x)) + 4*np.sin(f_2(x))

def bump_indicator(points, n=10, R=4):
    norm_points = np.linalg.norm(points, axis=1)
    f = np.zeros((points.shape[0]))
    f[norm_points<R]= np.exp(-1/(n*(1- (norm_points[norm_points<R]/R)**2)))
    return f


In [ ]:
r=4 #radius observation window
x = np.linspace(-r,r, 40)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
z_f_4 = bump_indicator(points)
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1)
ax.set_title("f_1")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2)
ax.set_title("f_2")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_2)
ax.set_title("f_3")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4)
ax.set_title("Bump indicator")
plt.show()

In [ ]:
from GPPY.numerical_integration import monte_carlo_integration
import statistics as stat

def var_lin_stat(f, points_list):
    mc_values = [monte_carlo_integration(points=p, f=f) for p in points_list]
    mean_mc = stat.mean(mc_values)
    std_mc = stat.stdev(mc_values)
    return mean_mc, std_mc
    

## d=3

In [ ]:
# F truncated is denoted F tilde, error notation
from structure_factor.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravity_point_process import GravityPointProcess
from structure_factor.spatial_windows import BallWindow

# Creat a poisson point process
d = 3
samples_nb = 30
nb_points =1e4
R=8 #simulation radius
r=4 #radius observation window
means_f1_push, stds_f1_push, means_f2_push, stds_f2_push= [], [], [], []
means_f3_push, stds_f3_push, means_f4_push, stds_f4_push = [], [], [], [] 
#using F trunc
means_f1_push_tilde, stds_f1_push_tilde, means_f2_push_tilde, stds_f2_push_tilde= [], [], [], []
means_f3_push_tilde, stds_f3_push_tilde, means_f4_push_tilde, stds_f4_push_tilde = [], [], [], [] 

#poisson pp
intensity = nb_points/simu_window.volume
simu_window = BallWindow(center=[0]*d, radius=R)
window = BallWindow(center=[0]*d, radius=r)
print("mean points_nb=", nb_points)
## list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp_big = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]

#Variance for poisson
poisson_pp = [p.restrict_to_window(window) for p in poisson_pp_big]
poisson_pts_list = [p.points for p in poisson_pp]
poisson_mean_nb_pt = stat.mean(p.shape[0] for p in poisson_pts_list)
## for f_1, f_2, f_3, f_4
mean_f1_poisson, std_f1_poisson = var_lin_stat(f_1, poisson_pts_list)
mean_f2_poisson, std_f2_poisson = var_lin_stat(f_2, poisson_pts_list)
mean_f3_poisson, std_f3_poisson = var_lin_stat(f_3, poisson_pts_list)
mean_f4_poisson, std_f4_poisson = var_lin_stat(bump_indicator, poisson_pts_list)

#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
epsilon_0 = gpp_pp[0].epsilon
print("d=", d, ",Nb samples=", samples_nb, ",Epsilon critical=", epsilon_0)
epsilons = [-0.05, -0.03, -0.01, -epsilon_0, -0.004, -0.001, -0.0005,
            0.001, 0.004, 0.005, epsilon_0, 0.006, 0.007, 0.008, 
            0.009, 0.01, 0.04, 0.07]
t=0
print("----------------------------------------------")
for epsilon in epsilons:
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
        
        #using F
        push_pp_big = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1,
                                             correction=True) for g in gpp_pp]
        push_pp = [g.restrict_to_window(window) for g in push_pp_big]
        push_pts_list = [g.points for g in push_pp]
        push_mean_nb_pt = stat.mean(g.shape[0] for g in push_pts_list)
        
        if t==0:
            time_end = time.time() - time_start
            print("start_time", int(time_end/60), "min", time_end%60, "s")
            print("----------------------------------------------")
        
        #using F tilde
        push_pp_big_tilde = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1,
                                                    p=r, correction=False) for g in gpp_pp]
        push_pp_tilde = [g.restrict_to_window(window) for g in push_pp_big_tilde]
        push_pts_list_tilde = [g.points for g in push_pp_tilde]
        push_mean_nb_pt_tilde = stat.mean(g.shape[0] for g in push_pts_list_tilde)
        
        if t==0:
            time_end2 = time.time() - time_end
            print("start_time", int(time_end2/60), "min", time_end2%60, "s")
            print("----------------------------------------------")
            
        #plot 
        fig, axis = plt.subplots(1, 3, figsize=(15, 5))
        push_pp_big[0].plot(window=simu_window, axis= axis[0], c="k", s=0.2)
        push_pp[0].plot(window=simu_window, axis= axis[0], c="b", s=0.2)
        push_pp_big_tilde[0].plot(window=simu_window, axis= axis[1], c="k", s=0.2)
        push_pp_tilde[0].plot(window=simu_window, axis= axis[1], c="b", s=0.2)
        poisson_pp_big[0].plot(window=simu_window, axis= axis[2], c="k", s=0.2)
        poisson_pp[0].plot(window=simu_window, axis= axis[2], c="b", s=0.2)
        plt.show()
        
        # mean number of points in the restricted window
        print("mean nb_pts poisson", poisson_mean_nb_pt, 
              "mean nb_pts push", push_mean_nb_pt,
              "mean nb_pts push tilde", push_mean_nb_pt_tilde
             )
        # variance for push
        ## f_1
        mean_f, std_f = var_lin_stat(f_1, push_pts_list)
        means_f1_push.append(mean_f)
        stds_f1_push.append(std_f)
        ## f_2
        mean_f, std_f = var_lin_stat(f_2, push_pts_list)
        means_f2_push.append(mean_f)
        stds_f2_push.append(std_f)
        ## f_3
        mean_f, std_f = var_lin_stat(f_3, push_pts_list)
        means_f3_push.append(mean_f)
        stds_f3_push.append(std_f)
        ## f_4
        mean_f, std_f = var_lin_stat(bump_indicator, push_pts_list)
        means_f4_push.append(mean_f)
        stds_f4_push.append(std_f)
        
        # variance for push tilde
        ## f_1
        mean_f, std_f = var_lin_stat(f_1, push_pts_list_tilde)
        means_f1_push_tilde.append(mean_f)
        stds_f1_push_tilde.append(std_f)
        ## f_2
        mean_f, std_f = var_lin_stat(f_2, push_pts_list_tilde)
        means_f2_push_tilde.append(mean_f)
        stds_f2_push_tilde.append(std_f)
        ## f_3
        mean_f, std_f = var_lin_stat(f_3, push_pts_list_tilde)
        means_f3_push_tilde.append(mean_f)
        stds_f3_push_tilde.append(std_f)
        ## f_4
        mean_f, std_f = var_lin_stat(bump_indicator, push_pts_list_tilde)
        means_f4_push_tilde.append(mean_f)
        stds_f4_push_tilde.append(std_f)
        
        print("FOR f_1:")
        print("std poisson", std_f1_poisson, ",std push", stds_f1_push)
        print("std poisson", std_f1_poisson, ",std push tilde", stds_f1_push_tilde)
        print("----------------------------------------------")
        
        print("FOR f_2:")
        print("std poisson", std_f2_poisson, ",std push", stds_f2_push)
        print("std poisson", std_f2_poisson, ",std push tilde", stds_f2_push_tilde)
        print("----------------------------------------------")
        
        print("FOR f_3:")
        print("std poisson f_3", std_f3_poisson, ",std push", stds_f3_push)
        print("std poisson f_3", std_f3_poisson, ",std push tilde", stds_f3_push_tilde)
        print("----------------------------------------------")
        
        print("FOR f_4:")
        print("std poisson f_4", std_f4_poisson, ",std push", stds_f4_push)
        print("std poisson f_4", std_f4_poisson, ",std push tilde", stds_f4_push_tilde)
        print("----------------------------------------------")
        
        t=1
time_final = time.time() - time_start
print("start_time", int(time_final/60), "min", time_final%60, "s")


In [ ]:
fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_2)
ax.set_title(r"$f_3$")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4)
ax.set_title(r"$f_4$")
plt.savefig("linear_stat_used_for_var_comp.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 4))
#f_1
ax[0].plot(epsilons, stds_f1_push, "k.", label=r"push $(\tilde{F})$" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].plot(epsilons, stds_f1_push_tilde, "b.", label=r"push $(F)$" )
ax[0].plot(epsilons, stds_f1_push_tilde, "b" )
ax[0].plot(0, std_f1_poisson, "r*", label="poisson")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push), ymax=max(stds_f1_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
#ax[0].set_title(r"$f_1$")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_2
ax[1].plot(epsilons, stds_f2_push,  "k.", label=r"push $(\tilde{F})$")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(epsilons, stds_f2_push_tilde,  "b.", label=r"push $(F)$")
ax[1].plot(epsilons, stds_f2_push_tilde,  "b")
ax[1].plot(0, std_f2_poisson, "r*", label="poisson")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push), ymax=max(stds_f2_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push,  "k.", label=r"push $(\tilde{F})$")
ax[2].plot(epsilons, stds_f3_push,  "k")
ax[2].plot(epsilons, stds_f3_push_tilde,  "b.", label=r"push $(F)$")
ax[2].plot(epsilons, stds_f3_push_tilde,  "b")
ax[2].plot(0, std_f3_poisson, "r*", label="poisson")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push), ymax=max(stds_f3_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")

ax[2].legend()
#ax[2].set_title(r"$f_3$")
ax[2].set_xlabel(r"$\epsilon$")
#ax[2].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[3].plot(epsilons, stds_f4_push,  "k.", label=r"push $(\tilde{F})$")
ax[3].plot(epsilons, stds_f4_push,  "k")
ax[3].plot(epsilons, stds_f4_push_tilde,  "b.", label=r"push $(F)$")
ax[3].plot(epsilons, stds_f4_push_tilde,  "b")
ax[3].plot(0, std_f4_poisson, "r*", label="poisson")
ax[3].vlines(epsilon_0, ymin=min(stds_f4_push), ymax=max(stds_f4_push), colors="grey", 
             linestyles='dashed', label=r"$\epsilon_0$")

ax[3].legend()
#ax[3].set_title(r"$f_3$")
ax[3].set_xlabel(r"$\epsilon$")
#ax[3].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
plt.savefig("variance_linear_stat_3d.pdf")
plt.show()

## d=2

In [ ]:
# F truncated is denoted F tilde, error notation
from structure_factor.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravity_point_process import GravityPointProcess
from structure_factor.spatial_windows import BallWindow

# Creat a poisson point process
d = 2
samples_nb = 30
nb_points =1e4
R=8 #simulation radius
r=4 #radius observation window
means_f1_push, stds_f1_push, means_f2_push, stds_f2_push= [], [], [], []
means_f3_push, stds_f3_push, means_f4_push, stds_f4_push = [], [], [], [] 
#using F trunc
means_f1_push_tilde, stds_f1_push_tilde, means_f2_push_tilde, stds_f2_push_tilde= [], [], [], []
means_f3_push_tilde, stds_f3_push_tilde, means_f4_push_tilde, stds_f4_push_tilde = [], [], [], [] 

#poisson pp
intensity = nb_points/simu_window.volume
simu_window = BallWindow(center=[0]*d, radius=R)
window = BallWindow(center=[0]*d, radius=r)
print("mean points_nb=", nb_points)
## list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp_big = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]

#Variance for poisson
poisson_pp = [p.restrict_to_window(window) for p in poisson_pp_big]
poisson_pts_list = [p.points for p in poisson_pp]
poisson_mean_nb_pt = stat.mean(p.shape[0] for p in poisson_pts_list)
## for f_1, f_2, f_3, f_4
mean_f1_poisson, std_f1_poisson = var_lin_stat(f_1, poisson_pts_list)
mean_f2_poisson, std_f2_poisson = var_lin_stat(f_2, poisson_pts_list)
mean_f3_poisson, std_f3_poisson = var_lin_stat(f_3, poisson_pts_list)
mean_f4_poisson, std_f4_poisson = var_lin_stat(bump_indicator, poisson_pts_list)

#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
epsilon_0 = gpp_pp[0].epsilon
print("d=", d, ",Nb samples=", samples_nb, ",Epsilon critical=", epsilon_0)
epsilons = [-0.05, -0.03, -0.01, -epsilon_0, -0.004, -0.001, -0.0005,
            0.001, 0.004, 0.005, epsilon_0, 0.006, 0.007, 0.008, 
            0.009, 0.01, 0.04, 0.07]
t=0
print("----------------------------------------------")
for epsilon in epsilons:
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
        
        #using F
        push_pp_big = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1,
                                             correction=True) for g in gpp_pp]
        push_pp = [g.restrict_to_window(window) for g in push_pp_big]
        push_pts_list = [g.points for g in push_pp]
        push_mean_nb_pt = stat.mean(g.shape[0] for g in push_pts_list)
        
        if t==0:
            time_end = time.time() - time_start
            print("start_time", int(time_end/60), "min", time_end%60, "s")
            print("----------------------------------------------")
        
        #using F tilde
        time_start = time.time()
        push_pp_big_tilde = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1,
                                                    p=r, correction=False) for g in gpp_pp]
        push_pp_tilde = [g.restrict_to_window(window) for g in push_pp_big_tilde]
        push_pts_list_tilde = [g.points for g in push_pp_tilde]
        push_mean_nb_pt_tilde = stat.mean(g.shape[0] for g in push_pts_list_tilde)
        
        if t==0:
            time_end2 = time.time() - time_start
            print("start_time", int(time_end2/60), "min", time_end2%60, "s")
            print("----------------------------------------------")
            
        #plot 
        fig, axis = plt.subplots(1, 3, figsize=(15, 5))
        push_pp_big[0].plot(window=simu_window, axis= axis[0], c="k", s=0.2)
        push_pp[0].plot(window=simu_window, axis= axis[0], c="b", s=0.2)
        push_pp_big_tilde[0].plot(window=simu_window, axis= axis[1], c="k", s=0.2)
        push_pp_tilde[0].plot(window=simu_window, axis= axis[1], c="b", s=0.2)
        poisson_pp_big[0].plot(window=simu_window, axis= axis[2], c="k", s=0.2)
        poisson_pp[0].plot(window=simu_window, axis= axis[2], c="b", s=0.2)
        plt.show()
        
        # mean number of points in the restricted window
        print("mean nb_pts poisson", poisson_mean_nb_pt, 
              "mean nb_pts push", push_mean_nb_pt,
              "mean nb_pts push tilde", push_mean_nb_pt_tilde
             )
        # variance for push
        ## f_1
        mean_f, std_f = var_lin_stat(f_1, push_pts_list)
        means_f1_push.append(mean_f)
        stds_f1_push.append(std_f)
        ## f_2
        mean_f, std_f = var_lin_stat(f_2, push_pts_list)
        means_f2_push.append(mean_f)
        stds_f2_push.append(std_f)
        ## f_3
        mean_f, std_f = var_lin_stat(f_3, push_pts_list)
        means_f3_push.append(mean_f)
        stds_f3_push.append(std_f)
        ## f_4
        mean_f, std_f = var_lin_stat(bump_indicator, push_pts_list)
        means_f4_push.append(mean_f)
        stds_f4_push.append(std_f)
        
        # variance for push tilde
        ## f_1
        mean_f, std_f = var_lin_stat(f_1, push_pts_list_tilde)
        means_f1_push_tilde.append(mean_f)
        stds_f1_push_tilde.append(std_f)
        ## f_2
        mean_f, std_f = var_lin_stat(f_2, push_pts_list_tilde)
        means_f2_push_tilde.append(mean_f)
        stds_f2_push_tilde.append(std_f)
        ## f_3
        mean_f, std_f = var_lin_stat(f_3, push_pts_list_tilde)
        means_f3_push_tilde.append(mean_f)
        stds_f3_push_tilde.append(std_f)
        ## f_4
        mean_f, std_f = var_lin_stat(bump_indicator, push_pts_list_tilde)
        means_f4_push_tilde.append(mean_f)
        stds_f4_push_tilde.append(std_f)
        
        print("FOR f_1:")
        print("std poisson", std_f1_poisson, ",std push", stds_f1_push)
        print("std poisson", std_f1_poisson, ",std push tilde", stds_f1_push_tilde)
        print("----------------------------------------------")
        
        print("FOR f_2:")
        print("std poisson", std_f2_poisson, ",std push", stds_f2_push)
        print("std poisson", std_f2_poisson, ",std push tilde", stds_f2_push_tilde)
        print("----------------------------------------------")
        
        print("FOR f_3:")
        print("std poisson f_3", std_f3_poisson, ",std push", stds_f3_push)
        print("std poisson f_3", std_f3_poisson, ",std push tilde", stds_f3_push_tilde)
        print("----------------------------------------------")
        
        print("FOR f_4:")
        print("std poisson f_4", std_f4_poisson, ",std push", stds_f4_push)
        print("std poisson f_4", std_f4_poisson, ",std push tilde", stds_f4_push_tilde)
        print("----------------------------------------------")
        
        t=1
time_final = time.time() - time_start
print("start_time", int(time_final/60), "min", time_final%60, "s")


In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 4))
#f_1
ax[0].plot(epsilons, stds_f1_push, "k.", label=r"push $(\tilde{F})$" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].plot(epsilons, stds_f1_push_tilde, "b.", label=r"push $(F)$" )
ax[0].plot(epsilons, stds_f1_push_tilde, "b" )
ax[0].plot(0, std_f1_poisson, "r*", label="poisson")
#ax[0].plot(epsilon_0, stds_f1_push[10], "m*", label=r"$\epsilon_0$")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push), ymax=max(stds_f1_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
#ax[0].set_title(r"$f_1$")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_2
ax[1].plot(epsilons, stds_f2_push,  "k.", label=r"push $(\tilde{F})$")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(epsilons, stds_f2_push_tilde,  "b.", label=r"push $(F)$")
ax[1].plot(epsilons, stds_f2_push_tilde,  "b")
ax[1].plot(0, std_f2_poisson, "r*", label="poisson")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push), ymax=max(stds_f2_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push,  "k.", label=r"push $(\tilde{F})$")
ax[2].plot(epsilons, stds_f3_push,  "k")
ax[2].plot(epsilons, stds_f3_push_tilde,  "b.", label=r"push $(F)$")
ax[2].plot(epsilons, stds_f3_push_tilde,  "b")
ax[2].plot(0, std_f3_poisson, "r*", label="poisson")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push), ymax=max(stds_f3_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")

ax[2].legend()
#ax[2].set_title(r"$f_3$")
ax[2].set_xlabel(r"$\epsilon$")
#ax[2].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[3].plot(epsilons, stds_f4_push,  "k.", label=r"push $(\tilde{F})$")
ax[3].plot(epsilons, stds_f4_push,  "k")
ax[3].plot(epsilons, stds_f4_push_tilde,  "b.", label=r"push $(F)$")
ax[3].plot(epsilons, stds_f4_push_tilde,  "b")
ax[3].plot(0, std_f4_poisson, "r*", label="poisson")
ax[3].vlines(epsilon_0, ymin=min(stds_f4_push), ymax=max(stds_f4_push), colors="grey", 
             linestyles='dashed', label=r"$\epsilon_0$")

ax[3].legend()
#ax[3].set_title(r"$f_3$")
ax[3].set_xlabel(r"$\epsilon$")
#ax[3].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
plt.savefig("variance_linear_stat_2d.pdf")
plt.show()